# 2. 전처리(Preprocessing)
- pdf, ppt로부터 불러온 텍스트 데이터를 다듬어주는 과정이다.
- 현재 쓰인 텍스트들은 줄바꿈, 불필요한 특수문자 등이 존재하기 때문에 전처리를 통해 다듬는 과정이 필요하다.

LLM을 사용하여 전처리를 진행하여, 완전한 문장으로 나누어질 수 있게 구분하려 한다.

## 2.1 모델 불러오기 (Ollama3-EEVE)

### 2.1.1 환경 설정
Ollama3 -EEVE 모델을 위한 `.gguf`파일 다운로드

`python magic method` - `%%bash` 를 사용하여 다운받을 수 있다.
- local_directory 부분에 모델을 다운받을 위치를 설정해주자.

```python
%%bash
huggingface-cli download \
  heegyu/EEVE-Korean-Instruct-10.8B-v1.0-GGUF \
  ggml-model-Q5_K_M.gguf \
  --local-dir {local_directory} \
  --local-dir-use-symlinks False
```
<br>
진행 환경에서는 `EEVE-Korean-Instruct-10.8B-v1.0-GGUF` 라는 폴더 생성 후 `.gguf` 파일을 넣어주었다.  
같은 위치에 확장자가 없는 `Modelfile`파일을 생성해주고 아래와 같이 작성해주었다.

#### Modelfile
```
FROM ggml-model-Q5_K_M.gguf

TEMPLATE """{{- if .System }}
<s>{{ .System }}</s>
{{- end }}
<s>Human:
{{ .Prompt }}</s>
<s>Assistant:
"""

SYSTEM """A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions."""

PARAMETER stop <s>
PARAMETER stop </s>
```

In [ ]:
%%bash
huggingface-cli download \
  heegyu/EEVE-Korean-Instruct-10.8B-v1.0-GGUF \
  ggml-model-Q5_K_M.gguf \
  --local-dir C:/Users/dm705/Study \
  --local-dir-use-symlinks False

### 2.1.2 파이썬에서 모델 불러오기(EEVE-Korean-10.8B)

In [7]:
"""
Requirements: pip install langchain
"""

from langchain_community.chat_models import ChatOllama   
from langchain_core.output_parsers import StrOutputParser 
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.callbacks.manager import CallbackManager

llm = ChatOllama(
    model="EEVE-Korean-10.8B:latest",
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
)

## 2. 프롬프트 템플릿 설정

In [72]:
import torch
if torch.cuda.is_available():
    torch.cuda.set_device(torch.device("cuda:0"))
    torch.cuda.empty_cache()

In [15]:
# 2. 프롬프트 템플릿 만들기
template = """
입력된 텍스트를 전처리하고 문장 단위로 나누어 주세요:
{text}

나뉘어진 문장별로 번호를 매겨 나누어 정리해 주십시오.
결과물 출력 외에 질문이나 답변은 하지 마십시오.
"""
prompt = ChatPromptTemplate.from_template(template)

# 3. 출력 파서 설정하기
output_parser = StrOutputParser()

# 4. LLMChain 만들기
llm_chain = prompt | llm | output_parser

## 3. 전처리 - 문장 구분하기 (using Ollama)

### 3.1. `*.txt` 텍스트 전처리 및 문장 구분하기

- 모델에 넣을 때
  - 모든 줄을 한 번에 읽으니, 출력 길이가 길지 않아서 막히는 것 같다.
  - 20줄씩 읽어서 텍스트를 전처리해주려고 한다.

In [74]:
def read_file_in_chunks(file_path, chunk_size=10):
    with open(file_path, 'r', encoding='utf-8') as file:
        while True:
            lines = []
            for _ in range(chunk_size):
                line = file.readline()
                if not line:
                    break
                lines.append(line.strip())
            if not lines:
                break
            yield lines

In [17]:
import os
directory = './dataset/'
files = os.listdir(directory)
txt_files = [file for file in files if file.endswith('.txt')]

for file_name in txt_files:
    file_path = directory + file_name
    
    result = []
    for chunk in read_file_in_chunks(file_path, chunk_size=20):
        result.append(llm_chain.invoke({"text":chunk}))


1. 배포일: 4월8일/ (총8매)
2. 담당부서: 보건복지부장애인정책과
3. 과장: 정충현
4. 전화번호: 02-2023-8190
5. 담당자: 이진희
6. 전화번호: 02-2023-8193
7. 제목: 제 33회 장애인의 날 교육자료, <장애인의 삶과 사회적 인식> 학습지도안
8. 기획 의도: 제 33회 장애인의 날을 맞이하여 학생들이 장애인을 이해하고 편견 없는 시선으로 바라보도록 지도할 수 있는 교육자료 제작 및 배포를 통해 장애인과 비장애인이 더불어 행복한 사회를 만드는 데 기여하고자 함.1. 든 데 스스로의 역할이 중요함을 자각하고, 배려를 실천할 수 있도록 독려합니다.
2. 활용 - 장애인 주간(4월 20일 ~26일)에 교육 권장. 
3. 주의: 교재 내 동영상 시청을 위하여 인터넷 연결 및 플래시 플레이어 최신 버전 업데이트가 되어있는지 확인 부탁드립니다. 
4. 학습지도안 - 주제: 장애인 삶과 사회적 인식. 
5. ○장애인에 대한 우리들의 인식과 편견 - 1p
6. ◆ 지도의 주안점: 평소 학생들이 장애인에 대해 갖고 있는 막연한 두려움이나 무관심 등을 자유롭게 논의할 수 있도록 유도합니다.
7. - 길이나 학교에서 장애인 만나면 어떤 생각이 드나요? 장애인 만나면 어떻게 대하나요?
8. - 같은 학급(학교)에 장애인 학생이 있는지? 친구를 놀릴 때 장애인 비하하는 표현을 쓰고 있지 않은지?
9. ○장애인이란? -2p
10. ◆ 지도의 주안점: 흔히 알려진 장애유형 외에도 다양한 원인으로 불편을 겪는 사람들이 있다는 사실을 알리고, 우리 모두가 각자 개성 있게 조금씩 다르게 태어난 것과 같이 장애인도 존중받아야 하는 존재임을 이해할 수 있도록 소개합니다.
11. - 2 -
12. ○장애인이란? -2p
13. ◆ 지도의 주안점: 흔히 알려진 장애유형 외에도 다양한 원인으로 불편을 겪는 사람들이 있다는 사실을 알리고, 우리 모두가 각자 개성 있게 조금씩 다르게 태어난 것과 같이 장애인도 존중받아야 하는 존재임을 이해할 수 있도록 소개합니다.. 장

In [23]:
pp(result[0])

('1. 배포일: 4월8일/ (총8매)\n'
 '2. 담당부서: 보건복지부장애인정책과\n'
 '3. 과장: 정충현\n'
 '4. 전화번호: 02-2023-8190\n'
 '5. 담당자: 이진희\n'
 '6. 전화번호: 02-2023-8193\n'
 '7. 제목: 제 33회 장애인의 날 교육자료, <장애인의 삶과 사회적 인식> 학습지도안\n'
 '8. 기획 의도: 제 33회 장애인의 날을 맞이하여 학생들이 장애인을 이해하고 편견 없는 시선으로 바라보도록 지도할 수 있는 교육자료 '
 '제작 및 배포를 통해 장애인과 비장애인이 더불어 행복한 사회를 만드는 데 기여하고자 함.')


In [21]:
from pprint import pprint as pp
import re

for sentences in result[:2]:
    for sentence in sentences:
        cleaned_text = re.sub(r'\n$', '', sentence)
        cleaned_text = re.sub(r'\r$', '', cleaned_text)

('1. 장애인은 신체 일부나 정신적으로 제한이 있어 일상생활과 사회 생활에 어려움을 겪습니다.\r\n'
 '2. 다양한 종류의 장애가 있으며, 일부는 외부의 문제가 아니지만 여전히 영향을 미칩니다.\r\n'
 '3. 장애는 모든 사람이 독특하게 태어난 것처럼 존중받아야 합니다.\r\n'
 '4. 참고: 다양한 장애 유형과 특징들입니다.\r\n'
 '5. 장애인 날 소개 - 4페이지로 시작합니다.\r\n'
 '6. 지도 주제: 장애인 날의 탄생 배경 설명 및 2013년부터 2017년까지 진행되는 제4차 장애인정책종합계획 비전, "장애인과 '
 '비장애인이 함께 행복한 사회"를 학생들에게 이해시키는 것을 목표로 합니다.\r\n'
 '7. 매년 4월 20일은 장애인의 날로, 올해는 33회를 맞이했습니다.\r\n'
 "8. 장애인 인식 증진과 재활 의지를 고취하는 기념일로 시작되었으며, 민간단체가 1972년부터 개최해 오던 4월 20일 '재활의 날'을 "
 "이어 받아 1981년에 나라에서 정식 '장애인의 날'로 지정했습니다.\r\n"
 '9. 올해 장애인 날 공식 표어는 "우리의 편견으로부터 장애는"입니다.')


In [71]:
from pprint import pprint as pp
import re

cleaned_sentence_list = []
for sentences in result:
    sub_sentences = re.sub(r'\r', '', sentences) # 정규표현식으로 '\r'을 제거
    sub_sentence_split = sub_sentences.split('\n') # 모든 문장을 '\n'으로 스플릿
    sub_sentence_sequence = list(map(lambda x:x[3:].strip(), sub_sentence_split)) # 문장마다 붙은 번호를 지움
    cleaned_sentence = '\n\n'.join(sub_sentence_sequence) # 다시 문장을 이어붙여줌. 구분자는 '\n\n'
    cleaned_sentence_list.append(cleaned_sentence)
    
# for문이 끝나면 나눠진 문장들을 다시 하나로 통합
cleaned_text = '\n\n'.join(cleaned_sentence_list)
print(cleaned_text)

with open('./dataset/data1_preprocessed.txt', 'w', encoding='utf-8') as file:
    file.write(cleaned_text)



배포일: 4월8일/ (총8매)

담당부서: 보건복지부장애인정책과

과장: 정충현

전화번호: 02-2023-8190

담당자: 이진희

전화번호: 02-2023-8193

제목: 제 33회 장애인의 날 교육자료, <장애인의 삶과 사회적 인식> 학습지도안

기획 의도: 제 33회 장애인의 날을 맞이하여 학생들이 장애인을 이해하고 편견 없는 시선으로 바라보도록 지도할 수 있는 교육자료 제작 및 배포를 통해 장애인과 비장애인이 더불어 행복한 사회를 만드는 데 기여하고자 함.

든 데 스스로의 역할이 중요함을 자각하고, 배려를 실천할 수 있도록 독려합니다.

활용 - 장애인 주간(4월 20일 ~26일)에 교육 권장.

주의: 교재 내 동영상 시청을 위하여 인터넷 연결 및 플래시 플레이어 최신 버전 업데이트가 되어있는지 확인 부탁드립니다.

학습지도안 - 주제: 장애인 삶과 사회적 인식.

○장애인에 대한 우리들의 인식과 편견 - 1p

◆ 지도의 주안점: 평소 학생들이 장애인에 대해 갖고 있는 막연한 두려움이나 무관심 등을 자유롭게 논의할 수 있도록 유도합니다.

- 길이나 학교에서 장애인 만나면 어떤 생각이 드나요? 장애인 만나면 어떻게 대하나요?

- 같은 학급(학교)에 장애인 학생이 있는지? 친구를 놀릴 때 장애인 비하하는 표현을 쓰고 있지 않은지?

○장애인이란? -2p

◆ 지도의 주안점: 흔히 알려진 장애유형 외에도 다양한 원인으로 불편을 겪는 사람들이 있다는 사실을 알리고, 우리 모두가 각자 개성 있게 조금씩 다르게 태어난 것과 같이 장애인도 존중받아야 하는 존재임을 이해할 수 있도록 소개합니다.

- 2 -

○장애인이란? -2p

◆ 지도의 주안점: 흔히 알려진 장애유형 외에도 다양한 원인으로 불편을 겪는 사람들이 있다는 사실을 알리고, 우리 모두가 각자 개성 있게 조금씩 다르게 태어난 것과 같이 장애인도 존중받아야 하는 존재임을 이해할 수 있도록 소개합니다.

장애인은 신체 일부나 정신적으로 제한이 있어 일상생활과 사회 생활에 어려움을 겪습니다.



In [35]:
cleaned_text

'1. 든 데 스스로의 역할이 중요함을 자각하고, 배려를 실천할 수 있도록 독려합니다.\r\n2. 활용 - 장애인 주간(4월 20일 ~26일)에 교육 권장. \r\n3. 주의: 교재 내 동영상 시청을 위하여 인터넷 연결 및 플래시 플레이어 최신 버전 업데이트가 되어있는지 확인 부탁드립니다. \r\n4. 학습지도안 - 주제: 장애인 삶과 사회적 인식. \r\n5. ○장애인에 대한 우리들의 인식과 편견 - 1p\r\n6. ◆ 지도의 주안점: 평소 학생들이 장애인에 대해 갖고 있는 막연한 두려움이나 무관심 등을 자유롭게 논의할 수 있도록 유도합니다.\r\n7. - 길이나 학교에서 장애인 만나면 어떤 생각이 드나요? 장애인 만나면 어떻게 대하나요?\r\n8. - 같은 학급(학교)에 장애인 학생이 있는지? 친구를 놀릴 때 장애인 비하하는 표현을 쓰고 있지 않은지?\r\n9. ○장애인이란? -2p\r\n10. ◆ 지도의 주안점: 흔히 알려진 장애유형 외에도 다양한 원인으로 불편을 겪는 사람들이 있다는 사실을 알리고, 우리 모두가 각자 개성 있게 조금씩 다르게 태어난 것과 같이 장애인도 존중받아야 하는 존재임을 이해할 수 있도록 소개합니다.\r\n11. - 2 -\r\n12. ○장애인이란? -2p\r\n13. ◆ 지도의 주안점: 흔히 알려진 장애유형 외에도 다양한 원인으로 불편을 겪는 사람들이 있다는 사실을 알리고, 우리 모두가 각자 개성 있게 조금씩 다르게 태어난 것과 같이 장애인도 존중받아야 하는 존재임을 이해할 수 있도록 소개합니다.'

In [ ]:
df.head()

In [11]:
import pandas as pd
import os

with open('./dataset/data1.txt', 'r', encoding='utf-8') as f:
    context = f.read()
    result = llm_chain.invoke({"text": context})


# # 결과 저장할 리스트
# results = []

# # 각 텍스트에 대해 처리
# for text in df[text_column]:
#     result = llm_chain.run({"text": text})
#     results.append(result)

# # 결과를 데이터프레임에 추가
# df["processed_text"] = results

# # 결과를 새로운 CSV 파일로 저장
# output_csv_path = "output.csv"
# df.to_csv(output_csv_path, index=False)

1. 장애인을 이해하고 존중하는 사회적 태도와 환경을 조성하기 위해 노력해야 합니다.
2. 인원수, 연령대 등 다양한 상황을 고려하여 장애인 인식 교육 및 캠페인 프로그램을 마련합니다.
3. 학교 교과과정과 연계한 활동 및 자료 개발을 통해 학생들이 일상생활 속에서 장애인 인식을 향상할 수 있도록 지원합니다.
4. 장애인을 대상으로 한 비장애인의 배려와 존중이 필요한 점들을 이해하고, 그 필요성에 대해 토론하는 시간을 가집니다.
5. 다양한 장애 유형에 따른 에티켓을 교육하고, 이를 바탕으로 장애인과 원활하게 소통 및 협력할 수 있는 방법을 제시합니다.
6. 시각장애인, 청각장애인, 지적장애인, 지체장애인을 위한 구체적인 에티켓을 설명하고 관련 행동과 태도를 제안합니다.
7. “우리는 친구사이” 애니메이션과 인터렉티브 시네마 영상을 통해 장애인과의 이해와 협력을 독려하는 활동을 진행합니다.
8. 시각장애, 청각장애, 지체장애, 학습장애 체험 활동 등을 통해 장애인이 겪는 어려움에 대해 간접적으로 경험하고 공감할 수 있는 기회를 제공합니다.
9. “2013 장애인 인식개선 광고”를 감상하며 같은 꿈을 꾸던 동갑내기 발레리나의 이야기를 통해 장애인을 이해하고 지원할 필요성에 대해 논의합니다.
10. 학교 및 주변 시설을 점검하여 장애인 편의시설이 잘 갖춰져 있는지 확인하고, 이러한 시설의 중요성을 이해하도록 장려합니다.